In [1]:
import itertools
import os

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from torch.utils.data import DataLoader
from transformers import DetrConfig, DetrForObjectDetection

from detr_config import Config
from detr_dataset import collate_fn, InBreastDataset, detr_processor
from detr_model import DETRModel


The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.


In [2]:

# HyperParameters

hyperparameters = itertools.product(*[
    Config.BACKBONES,
    Config.NUM_QUERIES,
    Config.D_MODEL,
    Config.TRANSFORMER_LAYERS,
])


# Hyperparameter Search

for backbone, num_queries, d_model, transformer_layers in hyperparameters:
    print('(Num Queries, Dim model, Enc-Dec Layers): ', 
            f'({num_queries}, {d_model}, {transformer_layers})' )

    # Model Construction

    config = DetrConfig.from_pretrained(
        Config.CHECKPOINT,
        num_labels=2,
        id2label = {0:'Mass', 1: 'No-Mass'}, 
        label2id = {'Mass': 0, 'No-Mass': 1},
        num_queries = num_queries,
        d_model = d_model,
        num_head = 8,
        encoder_layers = transformer_layers,
        decoder_layers = transformer_layers,
        position_embedding_type  = 'sine',
        decoder_ffn_dim = 2048,
        encoder_ffn_dim = 2048,
        backbone=backbone
    )


    # Training with K-fold Cross Validation 

    for fold in range(1,11):
        print(f"Fold {fold + 1}")
        
        detr_model = DetrForObjectDetection.from_pretrained(
            Config.CHECKPOINT,
            config = config,
            ignore_mismatched_sizes=True
        )

        model = DETRModel(detr_model=detr_model)
        
        fold_dir = os.path.join(Config.DATASET, f'fold_{fold}')
        
        train_dataset = InBreastDataset(
            images_path = os.path.join(fold_dir, 'train'),
            processor=detr_processor
        )
        
        train_loader = DataLoader(
            dataset = train_dataset,
            batch_size = Config.BATCH_SIZE,
            collate_fn=collate_fn,
        )

        valid_dataset = InBreastDataset(
            images_path = os.path.join(fold_dir, 'valid'),
            processor=detr_processor
        )

        valid_loader = DataLoader(
            dataset = train_dataset,
            batch_size = Config.BATCH_SIZE,
            collate_fn=collate_fn,
        )

        checkpoint_callback = pl.callbacks.ModelCheckpoint(
            save_top_k = 1,
            save_last = True,
            monitor = "valid_loss",
            mode = "min"
        )

        early_stopping_callback = pl.callbacks.EarlyStopping(
            monitor = 'valid_loss',
            patience = 15
        )

        version = os.path.join(
            f'backbone={backbone.split(".")[0]}_queries={num_queries}_dmodel={d_model}_layers={transformer_layers}',
            f'fold_{fold+1}'
        )

        logger = pl.loggers.TensorBoardLogger(
            save_dir = './',
            version = version
        )

        trainer = Trainer(
            max_epochs = Config.EPOCHS, 
            log_every_n_steps = 5, 
            callbacks = [
                checkpoint_callback, 
                early_stopping_callback
            ],
            accelerator = Config.ACCELERATOR,
            logger = logger
        )
        
        trainer.fit(
            model, 
            train_dataloaders = train_loader, 
            val_dataloaders = valid_loader
        )

        break # Fold
    break # Hyperparameter


(Num Queries, Dim model, Enc-Dec Layers):  (100, 256, 6)
Fold 2


Some weights of the model checkpoint at facebook/detr-resnet-50 were not used when initializing DetrForObjectDetection: ['model.backbone.conv_encoder.model.layer3.2.bn2.running_mean', 'model.backbone.conv_encoder.model.layer1.1.bn2.weight', 'model.backbone.conv_encoder.model.layer2.3.bn1.weight', 'model.backbone.conv_encoder.model.layer1.0.downsample.1.weight', 'model.backbone.conv_encoder.model.layer4.2.bn3.running_mean', 'model.backbone.conv_encoder.model.layer2.2.bn3.running_var', 'model.backbone.conv_encoder.model.layer2.2.bn2.weight', 'model.backbone.conv_encoder.model.layer3.2.bn1.running_var', 'model.backbone.conv_encoder.model.layer2.0.bn3.running_var', 'model.backbone.conv_encoder.model.layer1.0.downsample.0.weight', 'model.backbone.conv_encoder.model.layer3.0.downsample.1.bias', 'model.backbone.conv_encoder.model.layer4.1.conv2.weight', 'model.backbone.conv_encoder.model.layer3.3.bn2.running_mean', 'model.backbone.conv_encoder.model.layer3.3.conv2.weight', 'model.backbone.con

loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name       | Type                   | Params
------------------------------------------------------
0 | detr_model | DetrForObjectDetection | 21.2 M
------------------------------------------------------
21.2 M    Trainable params
0         Non-trainable params
21.2 M    Total params
84.642    Total estimated model params size (MB)


<generator object Module.named_parameters at 0x0000024E000ADA40>


Sanity Checking: 0it [00:00, ?it/s]

c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 16. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

c:\ProgramData\miniconda3\envs\detr-env\Lib\site-packages\pytorch_lightning\utilities\data.py:76: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 11. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(
`Trainer.fit` stopped: `max_epochs=1` reached.
